In [53]:
import pandas as pd
import os, sys
import datetime
import json

In [4]:
          ### CONCAT FILES IN CSVs FOLDER ###

# files = [file_path for file_path in os.listdir('CSVs') if file_path.endswith('.csv')]

# df = pd.read_csv(os.path.join('CSVs', files[0]))

# for csv_path in files[1:]:
#     df_to_add = pd.read_csv(os.path.join('CSVs', csv_path))
#     df = pd.concat([df,df_to_add])
    
    
# df.reset_index(drop=True, inplace=True)

#df.to_csv('Kick(2021-2022).csv')

In [353]:
# COLs = ['country','location','category', 'blurb', 'slug', 'backers_count', 
#        'created_at','goal','launched_at','deadline','converted_pledged_amount',
#         'state', 'currency', 'staff_pick', 'static_usd_rate','usd_type']
    

# def wrangle(CSV_PATH):
#     df = pd.read_csv(CSV_PATH, usecols= COLs)
#     df.dropna(inplace=True)

#     #Reorder columns
#     df = df[['country','location','category', 'blurb', 'slug', 'backers_count', 
#         'created_at','launched_at','deadline','converted_pledged_amount','goal',
#         'state', 'currency',
#         'staff_pick', 'static_usd_rate','usd_type']]
    
#     # Dropping rows where state is = to 'live' or 'canceled'
#     df = df[df['state'].isin(['successful','failed'])]
    
#     # Convert str to dicts (category and location columns)
#     convert_to_dicts = lambda x: json.loads(x)
#     df['category'] = df['category'].apply(convert_to_dicts)
#     df['location'] = df['location'].apply(convert_to_dicts)
    
#     # Exctract name from dicts (category and location columns)
#     extract_name = lambda x: x['name']
#     df['category'] = df['category'].apply(extract_name)
#     df['location'] = df['location'].apply(extract_name)
    
#     # Timestamps to Dates
#     convert = lambda x: datetime.datetime.fromtimestamp(x)
#     df['created_at'] = df['created_at'].apply(convert)
#     df['launched_at'] = df['launched_at'].apply(convert)
#     df['deadline'] = df['deadline'].apply(convert)
      
#     # Rename
#     df.columns = ['country','location','category', 'blurb', 'slug', 'n_backers', 
#        'created_at','launch_date','deadline','pledged_$','goal',
#         'state', 'currency',
#           'staff_pick', 'usd_rate','usd_type']
    
#     # New Features 
#     df['launch_week_of_the_year'] = [i.week for i in df['launch_date']]
#     df['campaign_duration_in_days'] = [i.days for i in duration]
    
#     # Drop Cols
#     df.drop(columns=['created_at','launch_date','deadline','usd_type', 'usd_rate'], inplace=True)
    
#     return df

# df = wrangle('Kick(2021-2022).csv')

In [361]:
df = pd.read_csv('Kick(2021-2022)_wrangled.zip',compression = "zip")

In [362]:
df.head()

,Unnamed: 0,country,location,category,blurb,slug,n_backers,pledged_$,goal,state,currency,staff_pick,launch_week_of_the_year,campaign_duration_in_days
0,0,US,Burlington,Glass,I make these adorable planters myself. Custo...,i-would-like-to-produce-and-sell-my-lightbulb-...,2,31,2000.0,failed,USD,False,34,30
1,1,US,Vineyard,Events,"The Utah Valley Food Fest is a sampling event,...",utah-valley-food-fest-2019,188,3367,3000.0,successful,USD,False,31,16
2,2,BE,Antwerp,Comedy,Newly found cabaret project Bunny Burlesk Even...,bbe-cabaret-burlesque-show,12,380,300.0,successful,EUR,False,40,60
3,3,US,Oxnard,Children's Books,A children's book about a bear that discovers ...,you-are-loved-a-bear-odd-fruit-origin-story,89,4839,4500.0,successful,USD,True,53,21
4,4,US,New York,Musical,What happens when the queen of darkness decide...,wish-a-mystical-magical-murderful-musical,30,10703,10000.0,successful,USD,False,27,31


In [356]:
df.to_csv('Kick(2021-2022)_wrangled.csv')

In [294]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208215 entries, 0 to 223288
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   country           208215 non-null  object        
 1   location          208215 non-null  object        
 2   category          208215 non-null  object        
 3   blurb             208215 non-null  object        
 4   slug              208215 non-null  object        
 5   n_backers         208215 non-null  int64         
 6   created_at        208215 non-null  datetime64[ns]
 7   launch_date       208215 non-null  datetime64[ns]
 8   deadline          208215 non-null  datetime64[ns]
 9   pledged_$         208215 non-null  int64         
 10  goal              208215 non-null  float64       
 11  state             208215 non-null  object        
 12  state_changed_at  208215 non-null  datetime64[ns]
 13  currency          208215 non-null  object        
 14  spot

In [324]:
df['launch_week_of_the_year'] = [i.week for i in df['launch_date']]
df['duration_in_days'] = [i.days for i in duration]



In [315]:
duration = df['deadline'] - df['launch_date']

In [322]:
df['duration_in_days'] = [i.days for i in duration]

In [325]:
df.head()

,country,location,category,blurb,slug,n_backers,created_at,launch_date,deadline,pledged_$,goal,state,state_changed_at,currency,spotlight,staff_pick,usd_rate,usd_type,duration,launch_week
0,US,Burlington,Glass,I make these adorable planters myself. Custo...,i-would-like-to-produce-and-sell-my-lightbulb-...,2,2014-08-14 17:03:06,2014-08-20 14:16:23,2014-09-19 14:16:23,31,2000.0,failed,2014-09-19 14:16:23,USD,False,False,1.000000,domestic,30,34
1,US,Vineyard,Events,"The Utah Valley Food Fest is a sampling event,...",utah-valley-food-fest-2019,188,2019-07-17 22:31:20,2019-07-31 21:55:28,2019-08-17 02:01:00,3367,3000.0,successful,2019-08-17 02:01:00,USD,True,False,1.000000,domestic,16,31
2,BE,Antwerp,Comedy,Newly found cabaret project Bunny Burlesk Even...,bbe-cabaret-burlesque-show,12,2018-10-01 04:32:40,2018-10-02 05:02:05,2018-12-01 05:02:05,380,300.0,successful,2018-12-01 05:02:05,EUR,True,False,1.159643,international,60,40
3,US,Oxnard,Children's Books,A children's book about a bear that discovers ...,you-are-loved-a-bear-odd-fruit-origin-story,89,2020-12-13 19:21:05,2020-12-29 22:00:16,2021-01-19 22:30:00,4839,4500.0,successful,2021-01-19 22:30:00,USD,True,True,1.000000,domestic,21,53
4,US,New York,Musical,What happens when the queen of darkness decide...,wish-a-mystical-magical-murderful-musical,30,2015-06-28 10:42:17,2015-07-02 17:49:19,2015-08-02 19:00:00,10703,10000.0,successful,2015-08-02 19:00:16,USD,True,False,1.000000,international,31,27


In [291]:
test.days

16

In [271]:
df.nunique()

country                 25
location             10834
category               161
blurb               130836
slug                131581
n_backers             3114
created_at          131535
launch_date         131489
deadline            125114
pledged_$            29267
goal                  4541
state                    2
state_changed_at    125202
currency                15
spotlight                2
staff_pick               2
usd_rate             13715
usd_type                 2
dtype: int64

In [41]:
datetime.datetime.fromtimestamp(df['created_at'][1])

datetime.datetime(2018, 5, 29, 9, 56, 42)

In [34]:
df['created_at'][0].to_pydatetime()

AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'

In [71]:
d_test = pd.read_csv('CSVs/2022 (9).csv')

In [241]:
convert = lambda x: datetime.datetime.fromtimestamp(x)
df['created_at'] = df['created_at'].apply(convert)
# d_test['created_at'] = d_test['created_at'].apply(convert)
# d_test['launched_at'] = d_test['launched_at'].apply(convert)

In [242]:
df['created_at'] 

0        2014-08-14 17:03:06
1        2019-07-17 22:31:20
2        2018-10-01 04:32:40
3        2020-12-13 19:21:05
4        2015-06-28 10:42:17
                 ...        
223284   2015-07-28 19:00:33
223285   2015-07-01 22:01:36
223286   2015-07-15 12:23:38
223287   2015-07-02 07:11:45
223288   2017-07-26 08:49:18
Name: created_at, Length: 222754, dtype: datetime64[ns]

In [76]:
df['last_update_published_at'].value_counts()

1.592374e+09    1
Name: last_update_published_at, dtype: int64

In [70]:
(d_test['deadline'][1] - d_test['created_at'][2])

Timedelta('216 days 11:17:00')